In [ ]:
"""
==================================================
ML LEARNING JOURNEY - DAY 24
==================================================
Week: 4 of 24
Day: 24 of 168
Date: November 19, 2025
Topic: Tracking Optimization & People Counting
Overall Progress: 14.3%

Week 4: Detection & Tracking Foundation
✅ Day 22: Project Planning & Architecture (COMPLETED)
✅ Day 23: Multi-Object Tracking (DeepSORT) (COMPLETED)
🔄 Day 24: Tracking Optimization (TODAY!)
⬜ Day 25: Video Processing Pipeline
⬜ Day 26: Testing & Performance
⬜ Day 27: Code Cleanup & Modularization
⬜ Day 28: Week 4 Review

Progress: 43% (3/7 days)

==================================================
🎯 Week 4 Project: Security System - Detection & Tracking
- Optimize tracking parameters for security scenarios
- Implement people counting (entry/exit)
- Handle occlusions and re-identification
- Define zones for monitoring
- Achieve consistent 30 FPS performance

🎯 Today's Learning Objectives:
1. Tune DeepSORT parameters (max_age, n_init, IOU threshold)
2. Implement line-crossing detection for people counting
3. Handle occlusions robustly (maintain IDs through disappearance)
4. Create zone-based monitoring (restricted areas)
5. Count people entering and exiting specific areas
6. Optimize for different scenarios (crowded, sparse, occlusions)
7. Measure tracking accuracy and ID consistency

📚 Today's Structure:
   Part 1 (2h): Parameter Tuning & Occlusion Handling
   Part 2 (2h): People Counting System
   Part 3 (2h): Zone-Based Monitoring
   Part 4 (1h): Testing & Summary

🎯 SUCCESS CRITERIA:
   ✅ Tracking parameters optimized (tested scenarios)
   ✅ Occlusions handled (IDs maintained through disappearance)
   ✅ Line-crossing detection working (entry/exit counting)
   ✅ People counting accurate (±5% error)
   ✅ Zone monitoring implemented (restricted areas)
   ✅ Direction detection working (in vs out)
   ✅ Performance maintained (25-30 FPS)
   ✅ Ready for video processing pipeline (Day 25)
==================================================
"""

In [1]:
# ==================================================
# INSTALL REQUIRED LIBRARIES
# ==================================================

import subprocess
import sys

print("📦 Installing required libraries...")
print("⏱️  This should be quick (most already installed)...\n")

packages = [
    'ultralytics',
    'deep-sort-realtime',
    'opencv-python',
    'numpy',
    'pandas',
    'matplotlib',
    'scipy'
]

for package in packages:
    print(f"Checking {package}...")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', package, '-q'])

print("\n✅ All libraries ready!")

print("\n" + "=" * 80)

# ==================================================
# IMPORT LIBRARIES
# ==================================================

print("\n" + "=" * 80)
print("📚 IMPORTING LIBRARIES")
print("=" * 80)

# Standard libraries
import os
import time
from pathlib import Path
from collections import defaultdict, deque
import json

# Data science
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import distance

# Computer vision
import cv2

# Deep learning
from ultralytics import YOLO

# Tracking
from deep_sort_realtime.deepsort_tracker import DeepSort

print("\n✅ All libraries imported successfully!")
print("\n📊 Library versions:")
print(f"   • OpenCV: {cv2.__version__}")
print(f"   • NumPy: {np.__version__}")
print(f"   • Pandas: {pd.__version__}")
print("   • Ultralytics: Installed ✓")
print("   • DeepSORT: Installed ✓")

print("=" * 80)

📦 Installing required libraries...
⏱️  This should be quick (most already installed)...

Checking ultralytics...
Checking deep-sort-realtime...
Checking opencv-python...
Checking numpy...
Checking pandas...
Checking matplotlib...
Checking scipy...

✅ All libraries ready!


📚 IMPORTING LIBRARIES



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/daneaudrey/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/daneaudrey/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

✅ All libraries imported successfully!

📊 Library versions:
   • OpenCV: 4.12.0
   • NumPy: 2.0.2
   • Pandas: 2.3.3
   • Ultralytics: Installed ✓
   • DeepSORT: Installed ✓


In [2]:
print("\n" + "=" * 80)
print("📚 PART 1: PARAMETER TUNING & OCCLUSION HANDLING")
print("=" * 80)


📚 PART 1: PARAMETER TUNING & OCCLUSION HANDLING


In [3]:
# ==================================================
# EXERCISE 1.1: UNDERSTAND DEEPSORT PARAMETERS
# ==================================================

print("\n" + "=" * 80)
print("EXERCISE 1.1: Understanding DeepSORT Parameters")
print("=" * 80)

"""
📖 THEORY: DeepSORT Critical Parameters

The three most important parameters for tuning:

1. max_age (Default: 30)
   • How many frames to keep a track without detection
   • Higher = More robust to occlusions (but more false positives)
   • Lower = Faster deletion (but IDs lost easily)
   • Formula: max_age = expected_occlusion_time * FPS
   • Example: 1 second occlusion @ 30fps → max_age=30

2. n_init (Default: 3)
   • Consecutive detections needed to confirm a track
   • Higher = Fewer false tracks (but slower confirmation)
   • Lower = Faster confirmation (but more false positives)
   • Typically: 2-5 frames

3. max_iou_distance (Default: 0.7)
   • Threshold for matching based on bounding box overlap
   • IOU = Intersection over Union
   • Lower = Stricter matching (fewer ID switches)
   • Higher = More lenient (more matches, potential errors)
   • Range: 0.5-0.9

Additional Parameters:

4. max_cosine_distance (Default: 0.2)
   • Threshold for appearance matching
   • Based on ReID embedding similarity
   • Lower = Objects must look very similar
   • Higher = More lenient appearance matching

5. nn_budget (Default: 100)
   • Number of appearance features to keep per track
   • Higher = Better re-identification (but more memory)
   • Lower = Less memory (but worse re-ID)

==================================================

PARAMETER TUNING GUIDELINES:

Scenario: Crowded Areas (Mall, Station)
├── max_age: 20-30 (frequent occlusions)
├── n_init: 3-4 (reduce false positives)
├── max_iou_distance: 0.6-0.7 (strict matching)
└── Goal: Reduce ID switches in crowds

Scenario: Sparse Areas (Office Hallway)
├── max_age: 15-20 (people rarely hidden)
├── n_init: 2-3 (fast confirmation)
├── max_iou_distance: 0.7-0.8 (lenient)
└── Goal: Quick tracking, smooth experience

Scenario: High Occlusion (Factory, Warehouse)
├── max_age: 40-60 (objects frequently hidden)
├── n_init: 4-5 (confirm before showing ID)
├── max_iou_distance: 0.5-0.6 (very strict)
└── Goal: Maintain IDs through long occlusions

Scenario: Fast Moving Objects (Entrance/Exit)
├── max_age: 10-15 (people pass quickly)
├── n_init: 2 (immediate confirmation)
├── max_iou_distance: 0.7-0.8 (allow fast motion)
└── Goal: Count people before they leave frame

==================================================

IMPACT ON PERFORMANCE:

Speed Impact:
- max_age: Minimal (just checks counter)
- n_init: Minimal (just checks counter)
- max_iou_distance: None (threshold only)
- nn_budget: High budget = more memory, slightly slower
- Overall: Parameter tuning doesn't significantly affect FPS

Accuracy Impact:
- max_age too low → Lost tracks frequently
- max_age too high → False tracks persist
- n_init too low → Many false positives
- n_init too high → Slow to confirm real tracks
- max_iou_distance too low → Missed matches, ID switches
- max_iou_distance too high → Wrong matches

==================================================

TESTING METHODOLOGY:

1. Start with defaults (max_age=30, n_init=3, iou=0.7)
2. Record baseline metrics (ID switches, lost tracks)
3. Adjust ONE parameter at a time
4. Test on representative videos
5. Measure improvement
6. Iterate until optimal

Metrics to Track:
- ID switches per minute
- Average track length
- False positive tracks
- Lost tracks (premature deletion)
- Re-identification success rate
"""

print("""
📊 PARAMETER COMPARISON TABLE:

Parameter        | Default | Crowded | Sparse  | High Occlusion | Fast Moving
-----------------|---------|---------|---------|----------------|------------
max_age          |   30    |  20-30  |  15-20  |     40-60      |   10-15
n_init           |    3    |   3-4   |   2-3   |      4-5       |     2
max_iou_distance |  0.7    | 0.6-0.7 | 0.7-0.8 |    0.5-0.6     |  0.7-0.8
max_cosine_dist  |  0.2    |  0.15   |  0.25   |      0.15      |    0.25
nn_budget        |  100    |   100   |   50    |      150       |    50

Recommendation for Security System (Office/Factory):
- Start with: max_age=30, n_init=3, max_iou_distance=0.7
- Tune based on specific environment
- Test with real footage from deployment location
""")

print("\n✅ Exercise 1.1 Complete!")
print("=" * 80)


EXERCISE 1.1: Understanding DeepSORT Parameters

📊 PARAMETER COMPARISON TABLE:

Parameter        | Default | Crowded | Sparse  | High Occlusion | Fast Moving
-----------------|---------|---------|---------|----------------|------------
max_age          |   30    |  20-30  |  15-20  |     40-60      |   10-15
n_init           |    3    |   3-4   |   2-3   |      4-5       |     2
max_iou_distance |  0.7    | 0.6-0.7 | 0.7-0.8 |    0.5-0.6     |  0.7-0.8
max_cosine_dist  |  0.2    |  0.15   |  0.25   |      0.15      |    0.25
nn_budget        |  100    |   100   |   50    |      150       |    50

Recommendation for Security System (Office/Factory):
- Start with: max_age=30, n_init=3, max_iou_distance=0.7
- Tune based on specific environment
- Test with real footage from deployment location


✅ Exercise 1.1 Complete!


In [4]:
# ==================================================
# EXERCISE 1.2: CREATE PARAMETER TESTING FRAMEWORK
# ==================================================

print("\n" + "=" * 80)
print("EXERCISE 1.2: Create Parameter Testing Framework")
print("=" * 80)

"""
📖 THEORY: Systematic Parameter Testing

Why We Need a Framework:
- Consistent testing across parameter values
- Objective performance comparison
- Track metrics over time
- Make data-driven decisions

Testing Approach:
1. Define test video/scenario
2. Create multiple tracker configurations
3. Run each configuration
4. Collect metrics
5. Compare and select best

Metrics to Collect:
- Track count (total unique IDs seen)
- Average track duration (frames)
- ID switches (when same person gets new ID)
- FPS (frames per second)
- Lost tracks (tracks deleted prematurely)
"""

class TrackerConfig:
    """Configuration for DeepSORT tracker"""
    
    def __init__(self, name, max_age=30, n_init=3, max_iou_distance=0.7):
        self.name = name
        self.max_age = max_age
        self.n_init = n_init
        self.max_iou_distance = max_iou_distance
    
    def create_tracker(self):
        """Create DeepSORT tracker with this config"""
        return DeepSort(
            max_age=self.max_age,
            n_init=self.n_init,
            max_iou_distance=self.max_iou_distance,
            embedder="mobilenet",
            embedder_gpu=False
        )
    
    def __repr__(self):
        return f"{self.name}: max_age={self.max_age}, n_init={self.n_init}, iou={self.max_iou_distance}"

print("✅ Class created: TrackerConfig")
print("   • Purpose: Store and manage tracker configurations")
print("   • Methods: create_tracker(), __repr__()")

# Define test configurations
configs = [
    TrackerConfig("Default", max_age=30, n_init=3, max_iou_distance=0.7),
    TrackerConfig("High_Occlusion", max_age=50, n_init=4, max_iou_distance=0.6),
    TrackerConfig("Fast_Confirmation", max_age=20, n_init=2, max_iou_distance=0.75),
    TrackerConfig("Strict_Matching", max_age=30, n_init=3, max_iou_distance=0.5),
    TrackerConfig("Lenient_Matching", max_age=30, n_init=3, max_iou_distance=0.85),
]

print("\n📋 Test Configurations Created:")
print("=" * 80)
for i, config in enumerate(configs, 1):
    print(f"   {i}. {config}")

print("\n💡 Testing Strategy:")
print("   • Run each configuration on same video")
print("   • Collect metrics for comparison")
print("   • Select best for security system use case")

print("\n✅ Exercise 1.2 Complete!")
print("=" * 80)


EXERCISE 1.2: Create Parameter Testing Framework
✅ Class created: TrackerConfig
   • Purpose: Store and manage tracker configurations
   • Methods: create_tracker(), __repr__()

📋 Test Configurations Created:
   1. Default: max_age=30, n_init=3, iou=0.7
   2. High_Occlusion: max_age=50, n_init=4, iou=0.6
   3. Fast_Confirmation: max_age=20, n_init=2, iou=0.75
   4. Strict_Matching: max_age=30, n_init=3, iou=0.5
   5. Lenient_Matching: max_age=30, n_init=3, iou=0.85

💡 Testing Strategy:
   • Run each configuration on same video
   • Collect metrics for comparison
   • Select best for security system use case

✅ Exercise 1.2 Complete!


In [5]:
# ==================================================
# EXERCISE 1.3: OCCLUSION HANDLING TEST
# ==================================================

print("\n" + "=" * 80)
print("EXERCISE 1.3: Test Occlusion Handling")
print("=" * 80)

"""
📖 THEORY: Handling Occlusions in Tracking

What is Occlusion?
- When tracked object is temporarily hidden
- Person walks behind pillar
- Person temporarily leaves frame
- Person overlapped by another person

Why Occlusions are Challenging:
- No detection = no update to Kalman filter
- Predicted position becomes less accurate
- Risk of losing track ID
- Risk of assigning new ID when reappears

How DeepSORT Handles Occlusions:

1. Kalman Filter Prediction:
   • Continues predicting position during occlusion
   • Uses last known velocity
   • Prediction uncertainty increases over time

2. Track State Management:
   • Tracked → Lost (when no detection)
   • Lost track kept for max_age frames
   • If reappears: match by appearance + predicted position
   • If doesn't reappear: delete track

3. Appearance Descriptor:
   • ReID embedding stored for each track
   • When object reappears, match by appearance
   • Even if position prediction is off
   • Enables re-identification

Success Factors:
✓ max_age long enough for expected occlusion
✓ Appearance features distinctive enough
✓ Predicted position reasonably accurate
✓ No similar-looking objects in scene

==================================================

OCCLUSION SCENARIOS:

Short Occlusion (1-10 frames, <0.5 seconds):
├── Challenge: Low
├── Solution: Kalman prediction usually sufficient
├── max_age: 15-20 frames adequate
└── Success Rate: 95%+

Medium Occlusion (10-30 frames, 0.5-1 second):
├── Challenge: Moderate
├── Solution: Appearance matching critical
├── max_age: 30-40 frames needed
└── Success Rate: 80-90%

Long Occlusion (30+ frames, 1+ seconds):
├── Challenge: High
├── Solution: May require higher max_age + good appearance
├── max_age: 50-60+ frames
└── Success Rate: 60-80%

Permanent Occlusion (object leaves):
├── Challenge: N/A (expected behavior)
├── Solution: Track should be deleted after max_age
├── max_age: Balance between persistence and deletion
└── Goal: Delete cleanly, no ghost tracks
"""

def simulate_occlusion_scenario():
    """
    Simulate tracking through occlusion
    
    Demonstrates how max_age affects track persistence
    """
    print("\n🧪 OCCLUSION SIMULATION:")
    print("=" * 80)
    
    print("\nScenario: Person walks behind pillar")
    print("   • Visible: Frames 1-10")
    print("   • Occluded: Frames 11-25 (15 frames)")
    print("   • Reappears: Frame 26+")
    
    print("\n📊 Testing with different max_age values:")
    
    test_configs = [
        ("max_age=10", 10),
        ("max_age=20", 20),
        ("max_age=30", 30),
        ("max_age=50", 50),
    ]
    
    occlusion_duration = 15  # frames
    
    for name, max_age in test_configs:
        if max_age >= occlusion_duration:
            result = "✅ Track MAINTAINED"
            explanation = f"(max_age={max_age} > occlusion={occlusion_duration})"
        else:
            result = "❌ Track LOST"
            explanation = f"(max_age={max_age} < occlusion={occlusion_duration})"
        
        print(f"   {name:15} → {result:20} {explanation}")
    
    print("\n💡 Key Insight:")
    print("   • max_age must exceed expected occlusion duration")
    print("   • Too low: Frequent ID losses")
    print("   • Too high: Ghost tracks persist longer")
    print("   • Sweet spot: 1.5-2x typical occlusion duration")
    
    print("\n📈 Recommended max_age by Scenario:")
    print("   • Office (rare occlusions): 20-30 frames")
    print("   • Retail (moderate): 30-40 frames")
    print("   • Factory (frequent): 40-60 frames")
    print("   • Outdoor (variable): 30-50 frames")

simulate_occlusion_scenario()

print("\n✅ Exercise 1.3 Complete!")
print("=" * 80)


EXERCISE 1.3: Test Occlusion Handling

🧪 OCCLUSION SIMULATION:

Scenario: Person walks behind pillar
   • Visible: Frames 1-10
   • Occluded: Frames 11-25 (15 frames)
   • Reappears: Frame 26+

📊 Testing with different max_age values:
   max_age=10      → ❌ Track LOST         (max_age=10 < occlusion=15)
   max_age=20      → ✅ Track MAINTAINED   (max_age=20 > occlusion=15)
   max_age=30      → ✅ Track MAINTAINED   (max_age=30 > occlusion=15)
   max_age=50      → ✅ Track MAINTAINED   (max_age=50 > occlusion=15)

💡 Key Insight:
   • max_age must exceed expected occlusion duration
   • Too low: Frequent ID losses
   • Too high: Ghost tracks persist longer
   • Sweet spot: 1.5-2x typical occlusion duration

📈 Recommended max_age by Scenario:
   • Office (rare occlusions): 20-30 frames
   • Retail (moderate): 30-40 frames
   • Factory (frequent): 40-60 frames
   • Outdoor (variable): 30-50 frames

✅ Exercise 1.3 Complete!


In [6]:
print("\n" + "=" * 80)
print("🔢 PART 2: PEOPLE COUNTING SYSTEM")
print("=" * 80)


🔢 PART 2: PEOPLE COUNTING SYSTEM


In [7]:
# ==================================================
# EXERCISE 2.1: UNDERSTAND LINE-CROSSING DETECTION
# ==================================================

print("\n" + "=" * 80)
print("EXERCISE 2.1: Understanding Line-Crossing Detection")
print("=" * 80)

"""
📖 THEORY: Line-Crossing for People Counting

Concept:
- Define a virtual line in the video
- Track when objects cross the line
- Count direction (entering vs exiting)
- Maintain running totals

Why Line-Crossing?
✓ Simple and effective
✓ Works with any tracking system
✓ Direction-aware (in vs out)
✓ Handles multiple people simultaneously
✓ Industry standard approach

==================================================

LINE-CROSSING ALGORITHM:

Components Needed:
1. Line definition: Two points (x1, y1) and (x2, y2)
2. Track positions: Current and previous frame
3. Crossing detection: Did trajectory cross line?
4. Direction detection: Which way did they cross?

Mathematical Approach:

Method 1: Line Segment Intersection
- Check if line segment (prev_pos, curr_pos) intersects counting line
- Use cross product to determine intersection
- Determine direction using relative positions

Method 2: Signed Distance
- Calculate perpendicular distance from point to line
- Sign indicates which side of line
- Crossing = sign change between frames
- Direction = positive to negative vs negative to positive

==================================================

GEOMETRIC FORMULA:

For line from point A(x1, y1) to B(x2, y2)
And object center at point P(x, y)

Cross Product Method:
- v1 = (x2 - x1, y2 - y1)  # Line vector
- v2 = (x - x1, y - y1)    # Point vector
- cross = v1[0] * v2[1] - v1[1] * v2[0]
- cross > 0: Point on one side
- cross < 0: Point on other side
- cross = 0: Point on line

Crossing Detection:
- If sign(cross_prev) ≠ sign(cross_curr): CROSSED!
- Direction: cross_prev < 0 and cross_curr > 0 → Direction 1
- Direction: cross_prev > 0 and cross_curr < 0 → Direction 2

==================================================

LINE PLACEMENT STRATEGIES:

Entrance/Exit Monitoring:
┌────────────────────────┐
│                        │
│     ↓↓↓  ENTRANCE  ↑↑↑ │
│     ═══════════════════ │  ← Counting line
│                        │
│      MONITORED AREA    │
│                        │
└────────────────────────┘

Placement Tips:
✓ Place perpendicular to traffic flow
✓ Avoid areas with frequent stops
✓ Clear sightline (no obstructions)
✓ Adequate lighting
✓ Consider camera angle

Multiple Lines:
- Line 1: Main entrance
- Line 2: Secondary entrance
- Line 3: Exit-only door
- Total = sum of all lines

==================================================

COUNTING SCENARIOS:

Scenario 1: Single Entrance/Exit (Bidirectional)
- One line, count both directions
- IN: crosses line one way
- OUT: crosses line other way
- Net count = IN - OUT

Scenario 2: Separate Entrance/Exit
- Two lines (one per door)
- Line 1: Only count crossings in
- Line 2: Only count crossings out
- Total IN, Total OUT tracked separately

Scenario 3: Zone Counting
- Define polygon zone
- Count entries (any line crossing into zone)
- Count exits (any line crossing out of zone)
- Current occupancy = entries - exits

Scenario 4: Bidirectional Traffic
- Busy corridor, people going both ways
- Line in middle of corridor
- Count both directions
- Analyze traffic patterns

==================================================

HANDLING EDGE CASES:

Case 1: Person Lingers on Line
- Problem: Multiple crossings detected
- Solution: Cooldown period per track
- Only count once per track per X seconds

Case 2: Person Crosses Back and Forth
- Problem: Inflated count
- Solution: Track last crossing direction
- Only count if direction changes

Case 3: Multiple People Cross Simultaneously
- Problem: Accurate counting
- Solution: Track each person's ID separately
- Count each unique crossing

Case 4: Person Partially Crosses
- Problem: False positive
- Solution: Require full crossing
- Check both feet/bottom of bbox crossed

==================================================

PERFORMANCE CONSIDERATIONS:

Accuracy Factors:
- Tracking quality (ID consistency)
- Line placement (optimal location)
- Camera angle (perpendicular is best)
- Lighting conditions
- Occlusions at line

Expected Accuracy:
- Optimal conditions: 95-98%
- Normal conditions: 90-95%
- Challenging conditions: 85-90%

Common Errors:
- Missed counts: Track lost at line
- Double counts: ID switch at line
- Wrong direction: Tracking jitter near line
"""

print("""
📐 VISUAL REPRESENTATION:

Line-Crossing Detection:

Frame N-1:                Frame N:
┌─────────────┐          ┌─────────────┐
│             │          │             │
│    ●        │          │             │
│   P1        │          │      ●      │
│             │          │     P2      │
│ ═══════════ │ LINE     │ ═══════════ │ LINE
│             │          │             │
│             │          │             │
└─────────────┘          └─────────────┘

P1 above line (cross < 0)    P2 below line (cross > 0)
                    → CROSSING DETECTED! ✓

Direction Determination:
- P1 to P2: Negative to Positive → Direction: DOWN (or IN)
- P2 to P1: Positive to Negative → Direction: UP (or OUT)

==================================================

IMPLEMENTATION STEPS:

1. Define line coordinates
   line_start = (x1, y1)
   line_end = (x2, y2)

2. Track object centers
   track_positions[track_id] = [(x, y), ...]

3. Calculate cross product
   cross_prev = calculate_cross_product(prev_pos, line)
   cross_curr = calculate_cross_product(curr_pos, line)

4. Detect crossing
   if sign(cross_prev) != sign(cross_curr):
       crossing_detected = True

5. Determine direction
   if cross_prev < 0 and cross_curr > 0:
       direction = "ENTERING"
   elif cross_prev > 0 and cross_curr < 0:
       direction = "EXITING"

6. Update counters
   if direction == "ENTERING":
       count_in += 1
   elif direction == "EXITING":
       count_out += 1
   
   current_occupancy = count_in - count_out
""")

print("\n✅ Exercise 2.1 Complete!")
print("=" * 80)


EXERCISE 2.1: Understanding Line-Crossing Detection

📐 VISUAL REPRESENTATION:

Line-Crossing Detection:

Frame N-1:                Frame N:
┌─────────────┐          ┌─────────────┐
│             │          │             │
│    ●        │          │             │
│   P1        │          │      ●      │
│             │          │     P2      │
│ ═══════════ │ LINE     │ ═══════════ │ LINE
│             │          │             │
│             │          │             │
└─────────────┘          └─────────────┘

P1 above line (cross < 0)    P2 below line (cross > 0)
                    → CROSSING DETECTED! ✓

Direction Determination:
- P1 to P2: Negative to Positive → Direction: DOWN (or IN)
- P2 to P1: Positive to Negative → Direction: UP (or OUT)


IMPLEMENTATION STEPS:

1. Define line coordinates
   line_start = (x1, y1)
   line_end = (x2, y2)

2. Track object centers
   track_positions[track_id] = [(x, y), ...]

3. Calculate cross product
   cross_prev = calculate_cross_product(prev_

In [8]:
# ==================================================
# EXERCISE 2.2: IMPLEMENT LINE-CROSSING DETECTION
# ==================================================

print("\n" + "=" * 80)
print("EXERCISE 2.2: Implement Line-Crossing Detection")
print("=" * 80)

"""
📖 THEORY: Implementation Details

We'll implement:
1. Line definition (interactive or fixed)
2. Cross product calculation
3. Crossing detection logic
4. Direction determination
5. Count tracking
6. Visualization

Data Structures:
- line_coords: (x1, y1, x2, y2)
- track_history: {track_id: [(x, y), ...]}
- crossed_tracks: {track_id: last_direction}
- counters: {in: 0, out: 0, current: 0}
"""

class PeopleCounter:
    """
    People counting system using line-crossing detection
    """
    
    def __init__(self, line_start, line_end):
        """
        Initialize counter with line coordinates
        
        Args:
            line_start: (x, y) tuple for line start point
            line_end: (x, y) tuple for line end point
        """
        self.line_start = np.array(line_start)
        self.line_end = np.array(line_end)
        
        # Track positions history (last 2 positions per track)
        self.track_positions = defaultdict(lambda: deque(maxlen=2))
        
        # Tracks that have crossed (to prevent double counting)
        self.crossed_tracks = {}
        
        # Counters
        self.count_in = 0
        self.count_out = 0
        self.total_crossings = 0
        
        print(f"✅ PeopleCounter initialized")
        print(f"   Line: {line_start} → {line_end}")
    
    def _calculate_cross_product(self, point):
        """
        Calculate cross product to determine which side of line point is on
        
        Args:
            point: (x, y) tuple
            
        Returns:
            float: cross product (sign indicates side)
        """
        # Line vector
        line_vec = self.line_end - self.line_start
        
        # Point vector (from line start to point)
        point_vec = np.array(point) - self.line_start
        
        # Cross product (2D)
        cross = line_vec[0] * point_vec[1] - line_vec[1] * point_vec[0]
        
        return cross
    
    def update(self, tracks):
        """
        Update counter with new tracks
        
        Args:
            tracks: List of tracks from DeepSORT
            
        Returns:
            dict: Crossing events {track_id: direction}
        """
        crossings = {}
        
        for track in tracks:
            if not track.is_confirmed():
                continue
            
            track_id = track.track_id
            bbox = track.to_ltrb()
            x1, y1, x2, y2 = bbox
            
            # Get center point (bottom-center for better accuracy)
            center_x = int((x1 + x2) / 2)
            center_y = int(y2)  # Bottom of bounding box
            center = (center_x, center_y)
            
            # Add to position history
            self.track_positions[track_id].append(center)
            
            # Need at least 2 positions to detect crossing
            if len(self.track_positions[track_id]) < 2:
                continue
            
            # Get previous and current positions
            prev_pos = self.track_positions[track_id][0]
            curr_pos = self.track_positions[track_id][1]
            
            # Calculate cross products
            cross_prev = self._calculate_cross_product(prev_pos)
            cross_curr = self._calculate_cross_product(curr_pos)
            
            # Detect crossing (sign change)
            if np.sign(cross_prev) != np.sign(cross_curr) and cross_prev != 0:
                
                # Check if already counted this track recently
                if track_id in self.crossed_tracks:
                    last_direction = self.crossed_tracks[track_id]
                    
                    # Determine current direction
                    if cross_prev < 0 and cross_curr > 0:
                        curr_direction = "IN"
                    else:
                        curr_direction = "OUT"
                    
                    # Only count if direction changed (prevents oscillation)
                    if curr_direction == last_direction:
                        continue
                
                # Determine direction
                if cross_prev < 0 and cross_curr > 0:
                    direction = "IN"
                    self.count_in += 1
                else:
                    direction = "OUT"
                    self.count_out += 1
                
                self.total_crossings += 1
                self.crossed_tracks[track_id] = direction
                crossings[track_id] = direction
        
        return crossings
    
    def get_counts(self):
        """Get current counts"""
        return {
            'in': self.count_in,
            'out': self.count_out,
            'current': self.count_in - self.count_out,
            'total': self.total_crossings
        }
    
    def draw_line(self, frame):
        """
        Draw counting line on frame
        
        Args:
            frame: Input frame
            
        Returns:
            Annotated frame
        """
        annotated = frame.copy()
        
        # Draw line
        cv2.line(annotated, tuple(self.line_start.astype(int)), 
                tuple(self.line_end.astype(int)), (0, 255, 255), 3)
        
        # Draw endpoints
        cv2.circle(annotated, tuple(self.line_start.astype(int)), 8, (0, 255, 255), -1)
        cv2.circle(annotated, tuple(self.line_end.astype(int)), 8, (0, 255, 255), -1)
        
        # Add label
        mid_point = ((self.line_start + self.line_end) / 2).astype(int)
        cv2.putText(annotated, "COUNTING LINE", tuple(mid_point - [0, 15]),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        
        return annotated
    
    def draw_counts(self, frame):
        """
        Draw count information on frame
        
        Args:
            frame: Input frame
            
        Returns:
            Annotated frame
        """
        annotated = frame.copy()
        counts = self.get_counts()
        
        # Semi-transparent panel
        overlay = annotated.copy()
        cv2.rectangle(overlay, (10, 200), (350, 400), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.6, annotated, 0.4, 0, annotated)
        
        # Draw counts
        y_offset = 240
        cv2.putText(annotated, "PEOPLE COUNT", (20, y_offset),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        
        y_offset += 40
        cv2.putText(annotated, f"IN:  {counts['in']}", (20, y_offset),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        y_offset += 40
        cv2.putText(annotated, f"OUT: {counts['out']}", (20, y_offset),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        
        y_offset += 40
        cv2.putText(annotated, f"Current: {counts['current']}", (20, y_offset),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)
        
        return annotated

print("✅ Class created: PeopleCounter")
print("\n📊 Features:")
print("   • Line-crossing detection (cross product method)")
print("   • Direction-aware counting (IN vs OUT)")
print("   • Duplicate prevention (track last crossing)")
print("   • Oscillation handling (direction change required)")
print("   • Current occupancy calculation (IN - OUT)")
print("   • Visualization (line + counters)")

print("\n🧪 Testing PeopleCounter class...")

# Create test counter (horizontal line in middle of 720p frame)
test_counter = PeopleCounter(
    line_start=(100, 360),  # Left side, middle height
    line_end=(1180, 360)    # Right side, middle height
)

print(f"\n✅ Test counter created!")
print(f"   Initial counts: {test_counter.get_counts()}")

print("\n✅ Exercise 2.2 Complete!")
print("=" * 80)


EXERCISE 2.2: Implement Line-Crossing Detection
✅ Class created: PeopleCounter

📊 Features:
   • Line-crossing detection (cross product method)
   • Direction-aware counting (IN vs OUT)
   • Duplicate prevention (track last crossing)
   • Oscillation handling (direction change required)
   • Current occupancy calculation (IN - OUT)
   • Visualization (line + counters)

🧪 Testing PeopleCounter class...
✅ PeopleCounter initialized
   Line: (100, 360) → (1180, 360)

✅ Test counter created!
   Initial counts: {'in': 0, 'out': 0, 'current': 0, 'total': 0}

✅ Exercise 2.2 Complete!


In [9]:
# ==================================================
# EXERCISE 2.3: PEOPLE COUNTING DEMO
# ==================================================

print("\n" + "=" * 80)
print("EXERCISE 2.3: People Counting Demo with Webcam")
print("=" * 80)

"""
📖 THEORY: Complete Counting System

Integration Steps:
1. Initialize YOLO detector
2. Initialize DeepSORT tracker
3. Initialize PeopleCounter
4. For each frame:
   a. Detect with YOLO
   b. Track with DeepSORT
   c. Update counter
   d. Visualize everything
5. Display results

What to Observe:
- People get unique IDs
- IDs persist as they move
- Counter increments when crossing line
- Direction is detected (IN vs OUT)
- Current occupancy updates
- Visualization clear and informative
"""

print("""
🎥 COMPLETE PEOPLE COUNTING DEMO

Below is the full code for webcam people counting.
This integrates YOLO + DeepSORT + PeopleCounter!

Features:
✓ Real-time detection + tracking
✓ Line-crossing detection
✓ Direction-aware counting (IN/OUT)
✓ Current occupancy tracking
✓ Visual counting line
✓ Counter display panel
✓ FPS monitoring

📝 To run this demo:
1. Copy the code below to a new cell
2. Execute the cell
3. Your webcam will open
4. Walk across the yellow line to test counting!
5. Try going both directions (IN and OUT)
6. Press 'q' to quit

⚠️  Note: Adjust line position based on your camera view
         Default line is horizontal in middle of frame
""")

print("\n" + "=" * 80)
print("CODE: PEOPLE COUNTING DEMO")
print("=" * 80)

print("""
Copy this code to a new cell to run people counting:

-----------------------------------------------------------------------
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import time

# Load models
print("Loading models...")
model = YOLO('yolov8n.pt')
tracker = DeepSort(max_age=30, n_init=3, embedder="mobilenet", embedder_gpu=False)

# Initialize counter (adjust line position for your camera!)
# Line coordinates: (x1, y1) to (x2, y2)
# Default: horizontal line in middle of 640x480 frame
counter = PeopleCounter(
    line_start=(50, 240),   # Left side, middle height
    line_end=(590, 240)     # Right side, middle height
)

# Open webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Cannot open webcam!")
else:
    print("✅ Webcam opened!")
    print("🎥 Starting people counting... (Press 'q' to quit)")
    print("💡 Walk across the YELLOW line to test counting!")
    
    # FPS tracking
    fps_counter = 0
    start_time = time.time()
    fps = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # 1. YOLO Detection
        results = model.predict(frame, conf=0.5, classes=[0], verbose=False)
        detections = results[0].boxes
        
        # 2. Convert to DeepSORT format
        deepsort_input = []
        for box in detections:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf = float(box.conf[0])
            w = x2 - x1
            h = y2 - y1
            deepsort_input.append(([x1, y1, w, h], conf, 'person'))
        
        # 3. Update tracker
        tracks = tracker.update_tracks(deepsort_input, frame=frame)
        
        # 4. Update counter (detect crossings)
        crossings = counter.update(tracks)
        
        # 5. Visualize tracks
        for track in tracks:
            if not track.is_confirmed():
                continue
            
            track_id = track.track_id
            bbox = track.to_ltrb()
            x1, y1, x2, y2 = map(int, bbox)
            
            # Color: Green normally, Red if just crossed
            if track_id in crossings:
                color = (0, 0, 255)  # Red
                direction = crossings[track_id]
                label = f'ID: {track_id} ({direction})'
            else:
                color = (0, 255, 0)  # Green
                label = f'ID: {track_id}'
            
            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
            # Draw ID
            cv2.putText(frame, label, (x1, y1 - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # 6. Draw counting line
        frame = counter.draw_line(frame)
        
        # 7. Draw counter panel
        frame = counter.draw_counts(frame)
        
        # 8. Calculate FPS
        fps_counter += 1
        if fps_counter % 30 == 0:
            elapsed = time.time() - start_time
            fps = 30 / elapsed
            start_time = time.time()
        
        # 9. Display FPS
        cv2.putText(frame, f'FPS: {fps:.1f}', (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
        # 10. Show frame
        cv2.imshow('People Counting System', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    # Final statistics
    final_counts = counter.get_counts()
    print(f"\\n📊 Final Statistics:")
    print(f"   • People IN: {final_counts['in']}")
    print(f"   • People OUT: {final_counts['out']}")
    print(f"   • Current occupancy: {final_counts['current']}")
    print(f"   • Total crossings: {final_counts['total']}")
    print(f"   • Average FPS: {fps:.1f}")
-----------------------------------------------------------------------
""")

print("\n💡 What to Observe:")
print("   • Yellow horizontal line in middle of frame")
print("   • Each person gets unique green box + ID")
print("   • Box turns RED when crossing line")
print("   • Counter panel shows IN, OUT, Current")
print("   • IN increases when crossing one direction")
print("   • OUT increases when crossing other direction")
print("   • Current = IN - OUT (occupancy)")

print("\n🧪 Test Scenarios:")
print("   1. Walk across line left-to-right (should count IN)")
print("   2. Walk across line right-to-left (should count OUT)")
print("   3. Walk back and forth (alternates IN/OUT)")
print("   4. Multiple people cross together (each counted)")
print("   5. Person lingers on line (counted only once)")

print("\n✅ Exercise 2.3 Complete!")
print("=" * 80)


EXERCISE 2.3: People Counting Demo with Webcam

🎥 COMPLETE PEOPLE COUNTING DEMO

Below is the full code for webcam people counting.
This integrates YOLO + DeepSORT + PeopleCounter!

Features:
✓ Real-time detection + tracking
✓ Line-crossing detection
✓ Direction-aware counting (IN/OUT)
✓ Current occupancy tracking
✓ Visual counting line
✓ Counter display panel
✓ FPS monitoring

📝 To run this demo:
1. Copy the code below to a new cell
2. Execute the cell
3. Your webcam will open
4. Walk across the yellow line to test counting!
5. Try going both directions (IN and OUT)
6. Press 'q' to quit

⚠️  Note: Adjust line position based on your camera view
         Default line is horizontal in middle of frame


CODE: PEOPLE COUNTING DEMO

Copy this code to a new cell to run people counting:

-----------------------------------------------------------------------
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import time

# Load

In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import time
from collections import defaultdict, deque

# Load models
print("Loading models...")
model = YOLO('yolov8n.pt')

# Tracker without embedder
tracker = DeepSort(
    max_age=30,
    n_init=3,
    nms_max_overlap=1.0,
    embedder=None
)

# PeopleCounter class
class PeopleCounter:
    def __init__(self, line_start, line_end):
        self.line_start = np.array(line_start)
        self.line_end = np.array(line_end)
        self.track_positions = defaultdict(lambda: deque(maxlen=2))
        self.crossed_tracks = {}
        self.count_in = 0
        self.count_out = 0
        self.total_crossings = 0
    
    def _calculate_cross_product(self, point):
        line_vec = self.line_end - self.line_start
        point_vec = np.array(point) - self.line_start
        cross = line_vec[0] * point_vec[1] - line_vec[1] * point_vec[0]
        return cross
    
    def update(self, tracks):
        crossings = {}
        for track in tracks:
            if not track.is_confirmed():
                continue
            
            track_id = track.track_id
            bbox = track.to_ltrb()
            x1, y1, x2, y2 = bbox
            
            center_x = int((x1 + x2) / 2)
            center_y = int(y2)
            center = (center_x, center_y)
            
            self.track_positions[track_id].append(center)
            
            if len(self.track_positions[track_id]) < 2:
                continue
            
            prev_pos = self.track_positions[track_id][0]
            curr_pos = self.track_positions[track_id][1]
            
            cross_prev = self._calculate_cross_product(prev_pos)
            cross_curr = self._calculate_cross_product(curr_pos)
            
            if np.sign(cross_prev) != np.sign(cross_curr) and cross_prev != 0:
                if track_id in self.crossed_tracks:
                    last_direction = self.crossed_tracks[track_id]
                    if cross_prev < 0 and cross_curr > 0:
                        curr_direction = "IN"
                    else:
                        curr_direction = "OUT"
                    if curr_direction == last_direction:
                        continue
                
                if cross_prev < 0 and cross_curr > 0:
                    direction = "IN"
                    self.count_in += 1
                else:
                    direction = "OUT"
                    self.count_out += 1
                
                self.total_crossings += 1
                self.crossed_tracks[track_id] = direction
                crossings[track_id] = direction
        
        return crossings
    
    def get_counts(self):
        return {
            'in': self.count_in,
            'out': self.count_out,
            'current': self.count_in - self.count_out,
            'total': self.total_crossings
        }
    
    def draw_line(self, frame):
        annotated = frame.copy()
        cv2.line(annotated, tuple(self.line_start.astype(int)), 
                tuple(self.line_end.astype(int)), (0, 255, 255), 3)
        cv2.circle(annotated, tuple(self.line_start.astype(int)), 8, (0, 255, 255), -1)
        cv2.circle(annotated, tuple(self.line_end.astype(int)), 8, (0, 255, 255), -1)
        mid_point = ((self.line_start + self.line_end) / 2).astype(int)
        cv2.putText(annotated, "COUNTING LINE", tuple(mid_point - [0, 15]),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        return annotated
    
    def draw_counts(self, frame):
        annotated = frame.copy()
        counts = self.get_counts()
        overlay = annotated.copy()
        cv2.rectangle(overlay, (10, 200), (350, 400), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.6, annotated, 0.4, 0, annotated)
        
        y_offset = 240
        cv2.putText(annotated, "PEOPLE COUNT", (20, y_offset),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        y_offset += 40
        cv2.putText(annotated, f"IN:  {counts['in']}", (20, y_offset),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        y_offset += 40
        cv2.putText(annotated, f"OUT: {counts['out']}", (20, y_offset),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        y_offset += 40
        cv2.putText(annotated, f"Current: {counts['current']}", (20, y_offset),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)
        return annotated

# Initialize counter
counter = PeopleCounter(
    line_start=(50, 240),
    line_end=(590, 240)
)

# Open webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Cannot open webcam!")
else:
    print("✅ Webcam opened!")
    print("🎥 Starting people counting... (Press 'q' to quit)")
    print("💡 Walk across the YELLOW line to test counting!")
    
    fps_counter = 0
    start_time = time.time()
    fps = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # 1. YOLO Detection
        results = model.predict(frame, conf=0.5, classes=[0], verbose=False)
        detections = results[0].boxes
        
        # 2. Convert to DeepSORT format
        deepsort_input = []
        for box in detections:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf = float(box.conf[0])
            w = x2 - x1
            h = y2 - y1
            deepsort_input.append(([x1, y1, w, h], conf, 'person'))
        
        # 3. Update tracker with RANDOM embeddings (not zeros!)
        if len(deepsort_input) > 0:
            # Create random normalized embeddings
            dummy_embeddings = [np.random.rand(128).astype(np.float32) for _ in deepsort_input]
            tracks = tracker.update_tracks(deepsort_input, embeds=dummy_embeddings, frame=frame)
        else:
            tracks = []
        
        # 4. Update counter
        crossings = counter.update(tracks)
        
        # 5. Visualize tracks
        for track in tracks:
            if not track.is_confirmed():
                continue
            
            track_id = track.track_id
            bbox = track.to_ltrb()
            x1, y1, x2, y2 = map(int, bbox)
            
            if track_id in crossings:
                color = (0, 0, 255)
                direction = crossings[track_id]
                label = f'ID: {track_id} ({direction})'
            else:
                color = (0, 255, 0)
                label = f'ID: {track_id}'
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # 6. Draw counting line
        frame = counter.draw_line(frame)
        
        # 7. Draw counter panel
        frame = counter.draw_counts(frame)
        
        # 8. Calculate FPS
        fps_counter += 1
        if fps_counter % 30 == 0:
            elapsed = time.time() - start_time
            fps = 30 / elapsed
            start_time = time.time()
        
        # 9. Display FPS
        cv2.putText(frame, f'FPS: {fps:.1f}', (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
        # 10. Show frame
        cv2.imshow('People Counting System', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    # Final statistics
    final_counts = counter.get_counts()
    print(f"\n📊 Final Statistics:")
    print(f"   • People IN: {final_counts['in']}")
    print(f"   • People OUT: {final_counts['out']}")
    print(f"   • Current occupancy: {final_counts['current']}")
    print(f"   • Total crossings: {final_counts['total']}")
    print(f"   • Average FPS: {fps:.1f}")
    

Loading models...
✅ Webcam opened!
🎥 Starting people counting... (Press 'q' to quit)
💡 Walk across the YELLOW line to test counting!

📊 Final Statistics:
   • People IN: 0
   • People OUT: 0
   • Current occupancy: 0
   • Total crossings: 0
   • Average FPS: 7.1
